In [1]:
import re

import utils.utils as utils

In [2]:
data = utils.load_data_for_day(7)

In [3]:
dependencies = list([])

for line in data:
    match = re.search('Step ([A-Z]) must be finished before step ([A-Z]) can begin.', line)
    
    dependencies.append((match.group(1), match.group(2)))


In [4]:
def determine_remaining_steps(dependencies, work_order):
    return set(
        map(lambda dep: dep[0], dependencies)
    ).union(set(
        map(lambda dep: dep[1], dependencies)
    )).difference(work_order)

def find_steps_without_dep(dependencies, work_order):
    return list(
        determine_remaining_steps(dependencies, work_order)
            .difference(
                map(lambda dep: dep[1], determine_remaining_dep(dependencies, work_order))
            )
    )

def determine_remaining_dep(dependencies, work_order):
    return list(filter(
        lambda dep: dep[0] not in work_order, 
        dependencies
    ))

def find_next_step(dependencies, work_order):
    steps_without_dependencies = find_steps_without_dep(dependencies, work_order)
    steps_without_dependencies.sort()
    return steps_without_dependencies[0]

def determine_last_step(dependencies, work_order):
    return list(determine_remaining_steps(dependencies, work_order))[0]

In [5]:
work_order = list([])

while len(determine_remaining_steps(dependencies, work_order)) > 1:
    work_order.append(find_next_step(dependencies, work_order))
    
''.join(work_order) + str(determine_last_step(dependencies, work_order))

'ABDCJLFMNVQWHIRKTEUXOZSYPG'

In [6]:
NUMBER_OF_WORKERS = 1 + 4
BASE_WORK_TIME = 60

In [7]:
def determine_duration(step):
    return BASE_WORK_TIME + 1 + ord(step) - ord('A')


class Worker:
    def __init__(self, step=None, ready_time=0):
        self.__step = step
        self.__ready_time = ready_time
    
    def will_be_done(self, time):
        return self.__ready_time <= time
    
    def assign(self, step, time):
        self.__step = step
        self.__ready_time = time + determine_duration(step)
        
    def step(self):
        return self.__step
    
    def ready_time(self):
        return self.__ready_time
    
    def __repr__(self):
        return ('Worker(step=' + str(self.__step)
                + ', ready_time=' + str(self.__ready_time) 
                + ')')

    
def find_idle_workers(workers, time):
    return list(filter(
    lambda worker: worker.will_be_done(time),
    workers))

def steps_being_worked_on(workers):
    return set(map(
        lambda worker: worker.step(),
        workers
    ))

def assign_steps_to_workers(workers, dependencies, work_done, time):
    steps_without_dependencies = find_steps_without_dep(dependencies, work_done)
    availlable_steps = list(set(steps_without_dependencies).difference(steps_being_worked_on(workers)))
    availlable_steps.sort()
    idle = find_idle_workers(workers, time)
    [worker.assign(step, time) for worker, step in zip(idle, availlable_steps)]
    
def find_time_next_finished_step(workers, time):
    ready_times = map(
        lambda worker: worker.ready_time(),
        workers
    )
    future_ready_times = filter(
        lambda ready_time: ready_time > time,
        ready_times
    )
    return min(future_ready_times)

def gather_work_done(workers, time):
    idle = find_idle_workers(workers, time)
    return set(map(
        lambda worker: worker.step(),
        idle
    ))

In [8]:
def test_assignment(letter, time):
    worker = Worker()
    worker.assign(letter, time)
    print(letter + ': ' + str(worker))

test_assignment('A', 0)
test_assignment('B', 0)
test_assignment('Z', 0)
test_assignment('A', 10)

A: Worker(step=A, ready_time=61)
B: Worker(step=B, ready_time=62)
Z: Worker(step=Z, ready_time=86)
A: Worker(step=A, ready_time=71)


In [9]:
work_done = set([])
workers = [Worker() for i in range(NUMBER_OF_WORKERS)]
time = 0

while len(determine_remaining_steps(dependencies, work_done)) > 0:
    assign_steps_to_workers(workers, dependencies, work_done, time)
    time = find_time_next_finished_step(workers, time)
    work_done = work_done.union(gather_work_done(workers, time))
    
time

896